# 5.3 Daten filtern und gruppieren

Im vorherigen Kapitel haben wir Autos basierend auf ihrem Kilometerstand
gruppiert und visualisiert. Während diese Gruppierung automatisch im Hintergrund
stattfand, werden wir in diesem Kapitel lernen, wie wir direkt auf die
gruppierten Daten zugreifen und zusätzliche Analysen durchführen können.


## Lernziele

```{admonition} Lernziele
:class: goals
* Sie wissen, dass die Wahrheitswerte `True` (wahr)  oder `False` (falsch) in
  dem Datentyp **bool** gespeichert werden.
* Sie kennen die wichtigstens Vergleichsoperatoren (`<`, `<=`, `>`, `>=`, `==`,
  `!=`, `in`, `not in`) in Python.
* Sie können ein Pandas-DataFrame-Objekt nach einem Wert filtern.
* Sie können ein Pandas-DataFrame-Objekt mit den Methoden `groupby()` und
  `get_group()` gruppieren.
```


## Daten filtern

Im vorherigen Kapitel haben wir die Kilometerstände von Autos untersucht, die im
Jahr 2020 zugelassen und Mitte 2023 auf Autoscout24.de angeboten wurden. Bei der
Kategorisierung der Kilometerstände fiel auf, dass Fahrzeuge mit einer
Laufleistung von über 200000 km selten sind. Trotzdem beeinflusste dies die
Aufteilung in zehn gleichmäßige Gruppen, die von 0 km bis 435909 km reichten,
erheblich. Um eine genauere Analyse zu ermöglichen, wäre es sinnvoll, Fahrzeuge
mit einer Laufleistung von bis zu 200.000 km in den Fokus zu nehmen und die
Ausreißer auszuschließen. Daher widmen wir uns in diesem Kapitel der Filterung
von tabellarischen Datensätzen mithilfe von Pandas.

Zuerst laden wir den Datensatz und überprüfen den Inhalt.

In [1]:
import pandas as pd

data = pd.read_csv('autoscout24_DE_2020.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18566 entries, 0 to 18565
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Marke                 18566 non-null  object 
 1   Modell                18566 non-null  object 
 2   Farbe                 18546 non-null  object 
 3   Erstzulassung         18566 non-null  object 
 4   Jahr                  18566 non-null  int64  
 5   Preis (Euro)          18566 non-null  int64  
 6   Leistung (kW)         18552 non-null  float64
 7   Leistung (PS)         18552 non-null  float64
 8   Getriebe              18566 non-null  object 
 9   Kraftstoff            18566 non-null  object 
 10  Verbrauch (l/100 km)  15501 non-null  object 
 11  Verbrauch (g/km)      18566 non-null  object 
 12  Kilometerstand (km)   18566 non-null  float64
 13  Bemerkungen           18566 non-null  object 
dtypes: float64(3), int64(2), object(9)
memory usage: 2.0+ MB


Um die Autos mit einem Kilometerstand von bis zu 200000 km zu filtern,
vergleichen wir die entsprechende Spalte mit dem Wert 200000, indem wir den aus
der Mathematik bekannten Kleiner-gleich-Operators `<=` benutzen. Das Ergebnis
dieses Vergleichs speichern wir in der Variable `bedingung`.

In [2]:
bedingung = data['Kilometerstand (km)'] <= 200000

Aber was genau ist in der Variable `bedingung` enthalten? Schauen wir uns den
Datentyp an:

In [3]:
type(bedingung)

pandas.core.series.Series

Offensichtlich handelt es sich um ein Pandas-Series-Objekt. Für weitere
Informationen können wir die `.info()`-Methode aufrufen:

In [4]:
bedingung.info()

<class 'pandas.core.series.Series'>
RangeIndex: 18566 entries, 0 to 18565
Series name: Kilometerstand (km)
Non-Null Count  Dtype
--------------  -----
18566 non-null  bool 
dtypes: bool(1)
memory usage: 18.3 KB


In dem Series-Objekt sind 18566 Einträge vom Datentyp `bool` gespeichert. Diesen
Datentyp haben wir bisher nicht kennengelernt. Wir lassen die ersten fünf
Einträge ausgeben:

In [5]:
bedingung.head()

0    True
1    True
2    True
3    True
4    True
Name: Kilometerstand (km), dtype: bool

Sind alle Einträge mit dem Wert `True` gefüllt? Wie viele und vor allem welche
einzigartige Einträge gibt es in diesem Series-Objekt?

In [6]:
bedingung.unique()

array([ True, False])

Das Series-Objekt enthält nur `True` und `False`, was den Datentyp `bool`
charakterisiert. In diesem Datentyp können nur zwei verschiedene Werte
gespeichert werden, nämlich wahr (True) und falsch (False). Oft sind
Wahrheitswerte das Ergebnis eines Vergleichs, wie das folgende Code-Beispiel
zeigt:

In [7]:
x = 19
print(x  < 100)

True


In der Python-Programmierung wird der Datentyp bool oft verwendet, um
Programmcode zu verzweigen. Damit ist gemeint, dass Teile des Programms nur
durchlaufen und ausgeführt werden, wenn eine bestimmte Bedingung wahr (True)
ist. In dieser Vorlesung benutzen wir bool-Werte hauptsächlich zum Filtern von
Daten.

```{admonition} Welche Vergleichsoperatoren kennt Python
In Python können die mathematischen Vergleichsoperatoren in ihrer gewohnten
Schreibweise verwendet werden:
* `<` kleiner als
* `<=` kleiner als oder gleich 
* `>` größer
* `>=` größer als oder gleich
* `==` gleich (`=` ist der Zuweisungsoperator, nicht mit Gleichheit
  verwechseln!)
* `!=` ungleich 

Darüber hinaus kann mit `in` oder `not in` getestet werden, ob
ein Element in einer Liste ist oder eben nicht.
```

Aber was machen wir jetzt mit diesem Series-Objekt? Wir können es als Index
benutzen für den ursprünglichen Datensatz benutzen. Die Zeilen, in denen `True`
sthet, werden übernommen, die anderen verworfen.

In [8]:
autos_bis_200000km = data[bedingung]
autos_bis_200000km.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18525 entries, 0 to 18565
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Marke                 18525 non-null  object 
 1   Modell                18525 non-null  object 
 2   Farbe                 18505 non-null  object 
 3   Erstzulassung         18525 non-null  object 
 4   Jahr                  18525 non-null  int64  
 5   Preis (Euro)          18525 non-null  int64  
 6   Leistung (kW)         18511 non-null  float64
 7   Leistung (PS)         18511 non-null  float64
 8   Getriebe              18525 non-null  object 
 9   Kraftstoff            18525 non-null  object 
 10  Verbrauch (l/100 km)  15472 non-null  object 
 11  Verbrauch (g/km)      18525 non-null  object 
 12  Kilometerstand (km)   18525 non-null  float64
 13  Bemerkungen           18525 non-null  object 
dtypes: float64(3), int64(2), object(9)
memory usage: 2.1+ MB


Von den 18566 Autos wurden 18525 Autos übernommen. Ist denn die Filterung
geglückt? Wir verschaffen uns mit der `.describe()`-Methode einen schnellen
Überblick.

In [9]:
autos_bis_200000km.describe()

,Jahr,Preis (Euro),Leistung (kW),Leistung (PS),Kilometerstand (km)
count,18525.0,18525.000000,18511.000000,18511.000000,18525.000000
mean,2020.0,33513.975816,135.327697,184.079466,42251.647935
std,0.0,37458.475775,75.548942,102.680858,28741.442712
min,2020.0,5950.000000,4.000000,5.000000,0.000000
25%,2020.0,19990.000000,90.000000,122.000000,22720.000000
50%,2020.0,26489.000000,110.000000,150.000000,35722.000000
75%,2020.0,35490.000000,148.000000,201.000000,54422.000000
max,2020.0,959980.000000,596.000000,810.000000,199000.000000


Der maximale Eintrag für die Spalte `Kilometerstand (km)` ist 199000 km. Mit dem
Tilde-Operaot `~` können wir das Pandas-Series-Objekt `bedingung` in das
Gegenteil umwandeln. Damit können wir also die Autos, bei denen der Vergleich
`<= 200000` zu `False` ausgewertet wurde, herausfiltern.

In [10]:
autos_ab_200000km = data[~bedingung]
autos_ab_200000km.info()

<class 'pandas.core.frame.DataFrame'>
Index: 41 entries, 968 to 18517
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Marke                 41 non-null     object 
 1   Modell                41 non-null     object 
 2   Farbe                 41 non-null     object 
 3   Erstzulassung         41 non-null     object 
 4   Jahr                  41 non-null     int64  
 5   Preis (Euro)          41 non-null     int64  
 6   Leistung (kW)         41 non-null     float64
 7   Leistung (PS)         41 non-null     float64
 8   Getriebe              41 non-null     object 
 9   Kraftstoff            41 non-null     object 
 10  Verbrauch (l/100 km)  29 non-null     object 
 11  Verbrauch (g/km)      41 non-null     object 
 12  Kilometerstand (km)   41 non-null     float64
 13  Bemerkungen           41 non-null     object 
dtypes: float64(3), int64(2), object(9)
memory usage: 4.8+ KB


41 Autos, die 2020 zugelassen wurden, sollten Mitte 2023 mit einem
Kilometerstand von mehr als 200000 km verkauft werden. Schauen wir uns die
Statistik an.

In [11]:
autos_ab_200000km.describe()

,Jahr,Preis (Euro),Leistung (kW),Leistung (PS),Kilometerstand (km)
count,41.0,41.000000,41.000000,41.000000,41.000000
mean,2020.0,20958.926829,115.463415,156.975610,245033.951220
std,0.0,8955.785235,40.915216,55.554247,53031.305157
min,2020.0,8000.000000,56.000000,76.000000,201000.000000
25%,2020.0,15740.000000,85.000000,116.000000,209800.000000
50%,2020.0,19950.000000,103.000000,140.000000,227846.000000
75%,2020.0,26000.000000,140.000000,190.000000,259000.000000
max,2020.0,44900.000000,255.000000,347.000000,435909.000000


Und was sind das für Autos?

In [12]:
autos_ab_200000km.head(10)

,Marke,Modell,Farbe,Erstzulassung,Jahr,Preis (Euro),Leistung (kW),Leistung (PS),Getriebe,Kraftstoff,Verbrauch (l/100 km),Verbrauch (g/km),Kilometerstand (km),Bemerkungen
968,audi,Audi A6,silber,05/2020,2020,29100,170.0,231.0,Automatik,Hybrid (Elektro/Diesel),"5,9 l/100 km",187 g/km,215000.0,Avant 45 TDI quattro sport
1136,audi,Audi S4,weiß,08/2020,2020,34990,255.0,347.0,Automatik,Diesel,"6,3 l/100 km",165 g/km,227846.0,Avant 3.0 TDI quattro Optik-Paket
2265,bmw,BMW 530,weiß,05/2020,2020,32999,195.0,265.0,Automatik,Diesel,"5,9 l/100 km",156 g/km,209800.0,Baureihe 5 Touring 530 d xDrive M Sport
2340,bmw,BMW 218,orange,08/2020,2020,27500,100.0,136.0,Schaltgetriebe,Benzin,6 l/100 km,- (g/km),290000.0,218i Coupe M Sport
2726,citroen,Citroen Jumper,weiß,06/2020,2020,17999,103.0,140.0,Schaltgetriebe,Diesel,NaN,- (g/km),273999.0,L3 H4#Hoch/Lang#AT Motor erst 100.000km#
2732,citroen,Citroen Jumper,weiß,09/2020,2020,19975,103.0,140.0,Schaltgetriebe,Diesel,"6,5 l/100 km",170 g/km,215000.0,35 L4H2 Club Heavy BlueHD (AT Motor/Getr)
2734,citroen,Citroen Berlingo,weiß,09/2020,2020,11781,56.0,76.0,Schaltgetriebe,Diesel,"4,1 l/100 km",109 g/km,204377.0,"Kasten 1,5 BlueHDI Club M/L1 Klima Pdc"
2736,citroen,Citroen Berlingo,weiß,03/2020,2020,10980,75.0,102.0,Schaltgetriebe,Diesel,4 l/100 km,106 g/km,236400.0,Kasten BlueHDi 100 Club M/L1 Kühlkasten
2942,dacia,Dacia Dokker,grau,02/2020,2020,9500,70.0,95.0,Schaltgetriebe,Diesel,"4,2 l/100 km",111 g/km,237420.0,Express Comfort/1Hd./Scheckheftgepflegt
3045,dacia,Dacia Logan,silber,03/2020,2020,9600,70.0,95.0,Schaltgetriebe,Diesel,"3,6 l/100 km",92 g/km,217400.0,MCV II Kombi Comfort Navi Standh. PDC 8-fach b...


## Gruppieren

Eine Filterung nach Kilometerstand ermöglicht es uns, die Autos in zwei
Datensätze zu teilen: Autos mit bis zu 200000 km Laufleistung und jene mit mehr
als 200000 km (hierzu kann der Tilde-Operator (~) verwendet werden).

Wenden wir nun diese Technik an, um die Fahrzeuge basierend auf ihrer Marke zu
trennen. Ein Beispiel: Um alle "Audi"-Fahrzeuge zu extrahieren, verwenden wir
den folgenden Code:

In [13]:
bedingung_audi = data['Marke'] == 'audi'
audis = data[bedingung_audi]
audis.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1190 entries, 104 to 1293
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Marke                 1190 non-null   object 
 1   Modell                1190 non-null   object 
 2   Farbe                 1190 non-null   object 
 3   Erstzulassung         1190 non-null   object 
 4   Jahr                  1190 non-null   int64  
 5   Preis (Euro)          1190 non-null   int64  
 6   Leistung (kW)         1189 non-null   float64
 7   Leistung (PS)         1189 non-null   float64
 8   Getriebe              1190 non-null   object 
 9   Kraftstoff            1190 non-null   object 
 10  Verbrauch (l/100 km)  1010 non-null   object 
 11  Verbrauch (g/km)      1190 non-null   object 
 12  Kilometerstand (km)   1190 non-null   float64
 13  Bemerkungen           1190 non-null   object 
dtypes: float64(3), int64(2), object(9)
memory usage: 139.5+ KB


Diese Bedingung erfüllen 1.190 Autos. Der Gesamtdatensatz enthält jedoch 41
unterschiedliche Automarken. Es wäre ineffizient, für jede Marke eine separate
Filterung durchzuführen. Deshalb bietet Pandas die `.groupby()`-Methode, die es
erlaubt, die Daten automatisch nach den einzigartigen Einträgen einer Spalte zu
gruppieren:

In [14]:
autos_nach_marke = data.groupby('Marke')
type(autos_nach_marke)

pandas.core.groupby.generic.DataFrameGroupBy

Das Resultat ist eine spezielle Pandas-Datenstruktur namens `DataFrameGroupBy`.
Es sind nicht alle bisher bekannte Methoden auf dieses Objekt anwendbar, aber
beispielsweise die `.describe()`-Methode darf verwendet werden:

In [15]:
autos_nach_marke.describe()

Jahr                                                       \
                count    mean  std     min     25%     50%     75%     max   
Marke                                                                        
alfa-romeo       88.0  2020.0  0.0  2020.0  2020.0  2020.0  2020.0  2020.0   
aston-martin     16.0  2020.0  0.0  2020.0  2020.0  2020.0  2020.0  2020.0   
audi           1190.0  2020.0  0.0  2020.0  2020.0  2020.0  2020.0  2020.0   
bentley          41.0  2020.0  0.0  2020.0  2020.0  2020.0  2020.0  2020.0   
bmw            1039.0  2020.0  0.0  2020.0  2020.0  2020.0  2020.0  2020.0   
cadillac          1.0  2020.0  NaN  2020.0  2020.0  2020.0  2020.0  2020.0   
chevrolet        26.0  2020.0  0.0  2020.0  2020.0  2020.0  2020.0  2020.0   
citroen         446.0  2020.0  0.0  2020.0  2020.0  2020.0  2020.0  2020.0   
dacia           268.0  2020.0  0.0  2020.0  2020.0  2020.0  2020.0  2020.0   
dodge            77.0  2020.0  0.0  2020.0  2020.0  2020.0  2020.0  2020.0   
ferrari          54.0  2020.0  0.0  2020.0  2020.0  2020.0  2020.0  2020.0   
fiat            493.0  2020.0  0.0  2020.0  2020.0  2020.0  2020.0  2020.0   
ford           1632.0  2020.0  0.0  2020.0  2020.0  2020.0  2020.0  2020.0   
honda            68.0  2020.0  0.0  2020.0  2020.0  2020.0  2020.0  2020.0   
hyundai         431.0  2020.0  0.0  2020.0  2020.0  2020.0  2020.0  2020.0   
infiniti          3.0  2020.0  0.0  2020.0  2020.0  2020.0  2020.0  2020.0   
isuzu             3.0  2020.0  0.0  2020.0  2020.0  2020.0  2020.0  2020.0   
jaguar          317.0  2020.0  0.0  2020.0  2020.0  2020.0  2020.0  2020.0   
jeep            190.0  2020.0  0.0  2020.0  2020.0  2020.0  2020.0  2020.0   
kia             491.0  2020.0  0.0  2020.0  2020.0  2020.0  2020.0  2020.0   
lada              7.0  2020.0  0.0  2020.0  2020.0  2020.0  2020.0  2020.0   
lamborghini      50.0  2020.0  0.0  2020.0  2020.0  2020.0  2020.0  2020.0   
land-rover      415.0  2020.0  0.0  2020.0  2020.0  2020.0  2020.0  2020.0   
maserati         61.0  2020.0  0.0  2020.0  2020.0  2020.0  2020.0  2020.0   
mazda           250.0  2020.0  0.0  2020.0  2020.0  2020.0  2020.0  2020.0   
mercedes-benz  1454.0  2020.0  0.0  2020.0  2020.0  2020.0  2020.0  2020.0   
mini            621.0  2020.0  0.0  2020.0  2020.0  2020.0  2020.0  2020.0   
mitsubishi      105.0  2020.0  0.0  2020.0  2020.0  2020.0  2020.0  2020.0   
nissan          312.0  2020.0  0.0  2020.0  2020.0  2020.0  2020.0  2020.0   
opel           1538.0  2020.0  0.0  2020.0  2020.0  2020.0  2020.0  2020.0   
peugeot         361.0  2020.0  0.0  2020.0  2020.0  2020.0  2020.0  2020.0   
porsche         169.0  2020.0  0.0  2020.0  2020.0  2020.0  2020.0  2020.0   
renault         558.0  2020.0  0.0  2020.0  2020.0  2020.0  2020.0  2020.0   
rover             2.0  2020.0  0.0  2020.0  2020.0  2020.0  2020.0  2020.0   
seat           1492.0  2020.0  0.0  2020.0  2020.0  2020.0  2020.0  2020.0   
skoda          1518.0  2020.0  0.0  2020.0  2020.0  2020.0  2020.0  2020.0   
smart            48.0  2020.0  0.0  2020.0  2020.0  2020.0  2020.0  2020.0   
ssangyong        17.0  2020.0  0.0  2020.0  2020.0  2020.0  2020.0  2020.0   
toyota          783.0  2020.0  0.0  2020.0  2020.0  2020.0  2020.0  2020.0   
volkswagen     1359.0  2020.0  0.0  2020.0  2020.0  2020.0  2020.0  2020.0   
volvo           572.0  2020.0  0.0  2020.0  2020.0  2020.0  2020.0  2020.0   

              Preis (Euro)                 ... Leistung (PS)         \
                     count           mean  ...           75%    max   
Marke                                      ...                        
alfa-romeo            88.0   40263.840909  ...        280.00  510.0   
aston-martin          16.0  174516.750000  ...        551.00  725.0   
audi                1190.0   42584.115126  ...        310.00  810.0   
bentley               41.0  239720.682927  ...        635.00  635.0   
bmw                 1039.0   38951.122233  ...        265.00  625.0   
cadillac        

Für jede Automarke werden nun für jede Spalte mit metrischen (quantitativen)
Informationen die statistischen Kennzahlen ermittelt. Die entstehende Tabelle
ist etwas unübersichtlich. Besser ist daher, sich die statistischen Kennzahlen
einzeln ausgeben zu lassen. Im folgenden ermitteln wir die Mittelwerte der
metrischen Informationen nach Automarke. Damit tatsächlich auch nur die
metrischen Daten gemittelt werden, müssen wir als Argument noch zusätzlich
`numeric_only=True` setzen.

In [16]:
autos_nach_marke.mean(numeric_only=True)

,Jahr,Preis (Euro),Leistung (kW),Leistung (PS),Kilometerstand (km)
Marke,,,,,
alfa-romeo,2020.0,40263.840909,182.704545,248.318182,40671.568182
aston-martin,2020.0,174516.750000,406.562500,553.000000,18923.250000
audi,2020.0,42584.115126,181.142136,246.392767,50733.891597
bentley,2020.0,239720.682927,425.585366,578.658537,29706.804878
bmw,2020.0,38951.122233,176.198459,239.561657,51443.319538
cadillac,2020.0,74900.000000,313.000000,426.000000,51000.000000
chevrolet,2020.0,42018.346154,286.692308,389.846154,35102.769231
citroen,2020.0,22244.840807,94.712360,128.883146,45375.890135
dacia,2020.0,15726.611940,79.458955,108.317164,38283.585821


Eine sehr wichtige Methode der GroupBy-Datenstruktur ist die
`get_group()`-Methode. Damit können wir ein bestimmtes DataFrame-Objekt aus dem
GroupBy-Objekt extrahieren:

In [17]:
audis_alternativ = autos_nach_marke.get_group('audi')
audis_alternativ.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1190 entries, 104 to 1293
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Marke                 1190 non-null   object 
 1   Modell                1190 non-null   object 
 2   Farbe                 1190 non-null   object 
 3   Erstzulassung         1190 non-null   object 
 4   Jahr                  1190 non-null   int64  
 5   Preis (Euro)          1190 non-null   int64  
 6   Leistung (kW)         1189 non-null   float64
 7   Leistung (PS)         1189 non-null   float64
 8   Getriebe              1190 non-null   object 
 9   Kraftstoff            1190 non-null   object 
 10  Verbrauch (l/100 km)  1010 non-null   object 
 11  Verbrauch (g/km)      1190 non-null   object 
 12  Kilometerstand (km)   1190 non-null   float64
 13  Bemerkungen           1190 non-null   object 
dtypes: float64(3), int64(2), object(9)
memory usage: 139.5+ KB


In der Variablen `audis_alternativ` steckt nun der gleiche Datensatz wie in der
Variablen `audis`, den wir bereits durch das Filtern des ursprünglichen
Datensatzes extrahiert haben. 


## Zusammenfassung und Ausblick

Dieses Kapitel hat uns in die Technik des Datenfilterns eingeführt. Um
spezifische Einträge aus einem Datensatz basierend auf einem bestimmten Wert zu
extrahieren, nutzen wir Vergleichsoperationen und verwenden das resultierende
Series-Objekt als Index. Wenn das Ziel darin besteht, Daten anhand der
einzigartigen Werte einer Spalte zu gruppieren, dann ist die Kombination von
`.groupby()` und `.get_group()` oft der effizienteste Weg. Damit haben wir
unsere Einführung in die Datenexploration abgeschlossen, obwohl es noch viele
weitere Möglichkeiten gibt, die Daten zu erkunden. Im nächsten Kapitel starten
wir mit den Grundlagen des maschinellen Lernens und befassen uns mit der
linearen Regression.